In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/te-v2-model/model_epoch_92.pt
/kaggle/input/lsa64-matrix-10-classes-v1/padded_matrix_file.csv
/kaggle/input/lsa64-matrix-unseen-5/LSA64-matrix-unseen-5.csv


In [2]:
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import csv
import ast
import torch.nn.functional as F
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder
import random
from torch.optim import Adam
from torch.nn.functional import cross_entropy
import pandas as pd

In [3]:
# Your code for loading the data (unchanged)
input_file = "/kaggle/input/lsa64-matrix-10-classes-v1/padded_matrix_file.csv"

values = []
matrix_labels = []
num_rows = 0

with open(input_file, "r") as f_input:
    reader = csv.reader(f_input)
    for row in reader:
        row_values = []
        for i in range(len(row) - 1):
            column_value = ast.literal_eval(row[i])
            row_values.append(column_value)
        values.append(torch.tensor(row_values))
        matrix_labels.append(ast.literal_eval(row[-1]))
        num_rows += 1

In [4]:
class TransformerEncoder(nn.Module):
    def __init__(self, n_features, d_model=64, nhead=64, num_layers=1):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Linear(n_features, d_model)
        self.positional_encoding = self.generate_positional_encoding(d_model)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead), num_layers
        )

    def generate_positional_encoding(self, d_model, max_len=243):
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)

    def forward(self, x):
        x = self.embedding(x)
        x = x + self.positional_encoding[:, : x.size(1)]
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        return x

In [5]:
# Load the pre-trained transformer encoder
encoder = TransformerEncoder(114)
encoder.load_state_dict(torch.load("/kaggle/input/te-v2-model/model_epoch_92.pt"))
encoder.eval()

TransformerEncoder(
  (embedding): Linear(in_features=114, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
)

In [6]:
# Extract features
embeddings = encoder(torch.stack(values))
labels = np.array(matrix_labels)

In [7]:
def create_few_shot_task(embeddings, labels, n_way, k_shot, n_query):
    unique_labels = np.unique(labels)
    selected_labels = np.random.choice(unique_labels, n_way, replace=False)
    
    support_set = []
    query_set = []
    support_labels = []
    query_labels = []
    
    for label in selected_labels:
        class_embeddings = embeddings[labels == label]
        support_indices = np.random.choice(range(len(class_embeddings)), k_shot, replace=False)
        query_indices = np.random.choice(np.delete(range(len(class_embeddings)), support_indices), n_query, replace=False)
        
        support_set.append(class_embeddings[support_indices].detach().numpy())
        query_set.append(class_embeddings[query_indices].detach().numpy())
        support_labels.extend([label] * k_shot)
        query_labels.extend([label] * n_query)
    
    support_set = np.concatenate(support_set, axis=0)
    query_set = np.concatenate(query_set, axis=0)
    support_labels = np.array(support_labels)
    query_labels = np.array(query_labels)
    
    return support_set, query_set, support_labels, query_labels


In [8]:
# n-way k-shot parameters
n_way = 10
k_shot = 5
n_query = 5
n_episodes = 100

# Episodic evaluation
accuracies = []

for episode in range(n_episodes):
    support_set, query_set, support_labels, query_labels = create_few_shot_task(embeddings, labels, n_way, k_shot, n_query)

    # Compute the class prototypes using the support set
    support_class_prototypes = np.zeros((n_way, embeddings.shape[-1]))
    for idx, label in enumerate(np.unique(support_labels)):
        support_class_prototypes[idx] = support_set[support_labels == label].mean(axis=0)

    # Compute distances between the query instances and the class prototypes
    distances = np.linalg.norm(support_class_prototypes[:, None] - query_set, axis=-1)

    # Classification
    predicted_labels = np.argmin(distances, axis=0)
    predicted_labels = np.array([np.unique(support_labels)[label_idx] for label_idx in predicted_labels])

    # Evaluate the performance
    accuracy = accuracy_score(query_labels, predicted_labels)
    accuracies.append(accuracy)

mean_accuracy = np.mean(accuracies)
print(f"{n_way}-way {k_shot}-shot mean accuracy over {n_episodes} episodes: {mean_accuracy * 100:.2f}%")

10-way 5-shot mean accuracy over 100 episodes: 97.56%


In [9]:
# Create a new few-shot task for demonstration purposes
support_set, query_set, support_labels, query_labels = create_few_shot_task(embeddings, labels, n_way, k_shot, n_query)

# Compute the class prototypes using the support set
support_class_prototypes = np.zeros((n_way, embeddings.shape[-1]))
for idx, label in enumerate(np.unique(support_labels)):
    support_class_prototypes[idx] = support_set[support_labels == label].mean(axis=0)

# Run t-SNE for 2D visualization
tsne = TSNE(n_components=2, random_state=42)
all_embeddings = np.vstack((support_set, query_set, support_class_prototypes))
all_embeddings_2d = tsne.fit_transform(all_embeddings)

# Prepare the data for visualization
support_labels_str = [str(x) for x in support_labels.tolist()]
query_labels_str = [str(x) for x in query_labels.tolist()]
prototype_labels_str = [str(x) for x in set(support_labels.tolist())]

support_types = ['support'] * len(support_labels)
query_types = ['query'] * len(query_labels)
prototype_types = ['prototype'] * len(prototype_labels_str)

all_labels = support_labels_str + query_labels_str + prototype_labels_str
all_types = support_types + query_types + prototype_types

df = pd.DataFrame({'x': all_embeddings_2d[:, 0], 'y': all_embeddings_2d[:, 1], 'label': all_labels, 'type': all_types})

# Create the plot
fig = go.Figure()

# Add support set points
support_df = df[df['type'] == 'support']
fig.add_trace(go.Scatter(x=support_df['x'], y=support_df['y'], mode='markers', marker=dict(symbol='circle', size=8),
                         text=support_df['label'], customdata=support_df['type'], name='Support Set',
                         hovertemplate='Label: %{text}<br>Type: %{customdata}<extra></extra>'))

# Add query set points
query_df = df[df['type'] == 'query']
fig.add_trace(go.Scatter(x=query_df['x'], y=query_df['y'], mode='markers', marker=dict(symbol='star', size=10),
                         text=query_df['label'], customdata=query_df['type'], name='Query Set',
                         hovertemplate='Label: %{text}<br>Type: %{customdata}<extra></extra>'))

# Add prototype points
prototype_df = df[df['type'] == 'prototype']
fig.add_trace(go.Scatter(x=prototype_df['x'], y=prototype_df['y'], mode='markers', marker=dict(symbol='x', size=12),
                         text=prototype_df['label'], customdata=prototype_df['type'], name='Prototypes',
                         hovertemplate='Label: %{text}<br>Type: %{customdata}<extra></extra>'))

fig.update_layout(title='2D t-SNE Visualization of Embeddings',
                  xaxis_title='t-SNE Component 1',
                  yaxis_title='t-SNE Component 2')

fig.show()


/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [10]:
import plotly.graph_objs as go
import plotly.express as px

# Create a color map for the unique labels
unique_labels = np.unique(np.array(all_labels))
colors = px.colors.qualitative.Plotly[:len(unique_labels)]
color_map = dict(zip(unique_labels, colors))

# Update the DataFrame with colors
df['color'] = df['label'].map(color_map)

# Create the plot
fig = go.Figure()

# Loop through unique labels and plot each class separately
for label, color in color_map.items():
    class_df = df[df['label'] == label]

    # Add support set points
    support_df = class_df[class_df['type'] == 'support']
    fig.add_trace(go.Scatter(x=support_df['x'], y=support_df['y'], mode='markers', marker=dict(symbol='circle', size=8, color=color),
                             text=support_df['label'], customdata=support_df['type'], name=f'Support Set - {label}',
                             hovertemplate='Label: %{text}<br>Type: %{customdata}<extra></extra>', showlegend=False))

    # Add query set points
    query_df = class_df[class_df['type'] == 'query']
    fig.add_trace(go.Scatter(x=query_df['x'], y=query_df['y'], mode='markers', marker=dict(symbol='star', size=10, color=color),
                             text=query_df['label'], customdata=query_df['type'], name=f'Query Set - {label}',
                             hovertemplate='Label: %{text}<br>Type: %{customdata}<extra></extra>', showlegend=False))

    # Add prototype points
    prototype_df = class_df[class_df['type'] == 'prototype']
    fig.add_trace(go.Scatter(x=prototype_df['x'], y=prototype_df['y'], mode='markers', marker=dict(symbol='x', size=12, color=color),
                             text=prototype_df['label'], customdata=prototype_df['type'], name=f'Prototype - {label}',
                             hovertemplate='Label: %{text}<br>Type: %{customdata}<extra></extra>'))

fig.update_layout(title='2D t-SNE Visualization of Embeddings',
                  xaxis_title='t-SNE Component 1',
                  yaxis_title='t-SNE Component 2')

fig.show()
